In [1]:
from taxcalc import *
from bokeh.plotting import figure
from bokeh.io import show, output_notebook, push_notebook
from bokeh.models import HoverTool, ColumnDataSource, NumeralTickFormatter, Range1d, DataRange1d
from bokeh.models.widgets import Select
from bokeh.models.annotations import LabelSet
from bokeh.layouts import row, column
from notebookfunctions import distribution, percentile, weighted_sum

import pandas as pd
import numpy as np
import copy
output_notebook()

Loading BokehJS ...

In [2]:
start_year = 2014
end_year = 2027

In [3]:
ben_name_list = ['Medicare', 'Medicaid', 'Veterans Benefits', 'SNAP', 'SSI', 'Social Security']
ben_part_names = ['mcare_part', 'mcaid_part', 'vb_part', 'snap_part', 'ssi_part', 'ss_part']
years = [year for year in range(start_year, end_year + 1)]
participation_index = [i for i in range(1, 101)]
bin_axis = ['{}-{}%'.format(i - 10, i) for i in range(10, 110, 10)]

### Calculations for benefits plots

In [4]:
# calculator used to create the data for plotting
calc_plots = Calculator(records=Records.cps_constructor(), policy=Policy())

You loaded data for 2014.
Tax-Calculator startup automatically extrapolated your data to 2014.


In [5]:
# dictionaries that will hold plot data
benefit_total_years = {'Medicare': [], 'Medicaid': [],
                       'Veterans Benefits': [], 'SNAP': [],
                       'SSI': [], 'Social Security': [],
                       'year': [year for year in range(start_year, end_year)]}  # sum of each benefit by year
participation_rates = {}  # participation rates for each benefit by year
avg_ben_all = {}  # average benefit for everyone by AGI percentile
avg_ben_participants = {}  # average benefit by AGI percentile for just participants
ben_totals_bin = {}  # total benefits by year by AGI bin
ben_avg_bin = {}  # average benefits by year by AGI bin
# loop through each year to get data needed for benefits plots
for year in range(start_year, end_year + 1):
    calc_plots.advance_to_year(year)
    calc_plots.calc_all()
    # DataFrame to hold all the needed data
    ben_df = pd.DataFrame({'s006': calc_plots.records.s006,
                           'Medicare': calc_plots.records.mcare_ben,
                           'Medicaid': calc_plots.records.mcaid_ben,
                           'Veterans Benefits': calc_plots.records.vet_ben,
                           'SNAP': calc_plots.records.snap_ben,
                           'SSI': calc_plots.records.ssi_ben,
                           'Social Security': calc_plots.records.ss_ben,
                           'AGI': calc_plots.records.c00100})
    # create dummies for participation
    ben_df['mcare_part'] = np.where(ben_df['Medicare'] > 0, 1, 0)
    ben_df['mcaid_part'] = np.where(ben_df['Medicaid'] > 0, 1, 0)
    ben_df['vb_part'] = np.where(ben_df['Veterans Benefits'] > 0, 1, 0)
    ben_df['snap_part'] = np.where(ben_df['SNAP'] > 0, 1, 0)
    ben_df['ssi_part'] = np.where(ben_df['SSI'] > 0, 1, 0)
    ben_df['ss_part'] = np.where(ben_df['Social Security'] > 0, 1, 0)
    
    # initialize keys in certain dictionaries
    participation_rates[year] = {}
    avg_ben_all[year] = {}
    avg_ben_participants[year] = {}
    ben_totals_bin[year] = {}
    ben_avg_bin[year] = {}

    # loop through benefit names to add to dictionaries
    for i in range(0, len(ben_name_list)):
        ben = ben_name_list[i]
        ben_part = ben_part_names[i]
        # add weighted sum for each benefit to their list
        benefit_total_years[ben].append(weighted_sum(ben_df, ben))
        # add mean participation rates by benefit
        participation_rates[year][ben] = percentile(ben_df, ben_part, 100, 'AGI')
        participation_rates[year]['index'] = [i for i in range(1, 101)]
        # add average benefit by percentile for all
        avg_ben_all[year][ben] = percentile(ben_df, ben, 100, 'AGI')
        # add average benefit by percentile for participants
        sub_df = copy.deepcopy(ben_df[ben_df[ben_part] == 1])
        avg_ben_participants[year][ben] = percentile(sub_df, ben, 100, 'AGI')
        ben_totals_bin[year][ben] = percentile(ben_df, ben, 10, 'AGI', result_type='sum')
        ben_avg_bin[year][ben] = percentile(ben_df, ben, 10, 'AGI')

In [6]:
# add info to dictionaries as needed

benefit_total_years['year'] = years
#benefit_total_years['top'] = [i * 0 for i in years]

for year in range(start_year, end_year + 1):
    avg_ben_all[year]['index'] = participation_index
    avg_ben_participants[year]['index'] = participation_index

ben_totals_bin[2014]['index'] = bin_axis
ben_avg_bin[2014]['index'] = bin_axis
ben_totals_bin[2014]['top'] = ben_totals_bin[2014]['Medicaid']
ben_avg_bin[2014]['top'] = ben_avg_bin[2014]['Medicaid']

In [7]:
# create starting Column Data Sources
ben_total_cds = ColumnDataSource(benefit_total_years)
participation_rates_cds = ColumnDataSource(participation_rates[2014])
avg_ben_all_cds = ColumnDataSource(avg_ben_all[2014])
avg_ben_participants_cds = ColumnDataSource(avg_ben_participants[2014])
ben_totals_bin_cds = ColumnDataSource(ben_totals_bin[2014])
ben_avg_bin_cds = ColumnDataSource(ben_avg_bin[2014])

# Total Benefits by Year

In [8]:
# function to generate plots
def total_bar(benefit):
    total_fig = figure(title='Total {} Benefit by Year'.format(benefit),
                       y_range=(0.0, 1e12), x_range=DataRange1d())
    init_bar = total_fig.vbar(x='year', width=0.75, bottom=0, top=benefit, source=ben_total_cds)

    hover_format = '{$ 0.00 a}'
    hover = HoverTool(tooltips=[('Value', '@{}{}'.format(benefit, hover_format))])
    total_fig.add_tools(hover)

    total_fig.yaxis[0].formatter = NumeralTickFormatter(format='$0.00a')
    total_fig.xaxis.minor_tick_line_color = None
    total_fig.xaxis.axis_label = 'Year'
    total_fig.yaxis.axis_label = 'Total Benefit'
    
    return total_fig

In [9]:
# Add figures to a list
total_bar_list = []
for benefit in ben_name_list:
    total_bar_list.append(total_bar(benefit))

In [10]:
show(column(total_bar_list))

# Participation Rates by Program

In [11]:
# function for participation rate by program and year
def participation_rates_year(year, cds):
    part_fig = figure(title='Participation Rates by AGI Percentile for {}'.format(year))
    mcare_part = part_fig.line(x='index', y='Medicare', legend='Medicare ', line_width=2,
                               source=cds)
    mcaid_part = part_fig.line(x='index', y='Medicaid', legend='Medicaid ', line_width=2, color='red',
                               source=cds)
    ss_part = part_fig.line(x='index', y='Social Security', legend='Social Security ', line_width=2, color='green',
                            source=cds)
    snap_part = part_fig.line(x='index', y='SNAP', legend='SNAP ', line_width=2, color='orange',
                              source=cds)
    ssi_part = part_fig.line(x='index', y='SSI', legend='SSI ', line_width=2, color='purple',
                             source=participation_rates_cds)
    vb_part = part_fig.line(x='index', y='Veterans Benefits', legend='Veterans Benefits ', line_width=2, color='pink',
                            source=cds)
    hover_part = HoverTool(tooltips=[('Rate', '$y{0.0%}'), ('Percentile', '@index%')])
    part_fig.add_tools(hover_part)

    part_fig.yaxis.axis_label = 'Rate'
    part_fig.xaxis.axis_label = 'AGI Percentile'
    
    return part_fig

In [12]:
# create list participation rate CDS's
participation_cds_list = []
for year in range(start_year, end_year + 1):
    participation_cds_list.append(ColumnDataSource(participation_rates[year]))

In [13]:
participation_rate_plots = []
yr = 2014
for item in participation_cds_list:
    participation_rate_plots.append(participation_rates_year(yr, item))
    yr += 1

In [14]:
show(column(participation_rate_plots))

# Average Benefits for All

In [15]:
# function for average benefits
def avg_ben(title, cds):
    fig = figure(title=title)
    fig.line(x='index', y='Medicare', legend='Medicare ', line_width=2,
             source=cds)
    fig.line(x='index', y='Medicaid', legend='Medicaid ', line_width=2, color='red',
             source=cds)
    fig.line(x='index', y='Social Security', legend='Social Security ', line_width=2, color='green',
             source=cds)
    fig.line(x='index', y='SNAP', legend='SNAP ', line_width=2, color='orange',
             source=cds)
    fig.line(x='index', y='SSI', legend='SSI ', line_width=2, color='purple',
             source=cds)
    fig.line(x='index', y='Veterans Benefits', legend='Veterans Benefits ', line_width=2, color='pink',
             source=cds)
    hover_ab = HoverTool(tooltips=[('Benefit', '$y{$0,0.00}'), ('Percentile', '@index%')])
    fig.add_tools(hover_ab)

    fig.xaxis.axis_label = 'Percentile'
    fig.yaxis.axis_label = 'Average Benefit'
    
    return fig

In [16]:
# create list participation rate CDS's
avg_ben_all_cds_list = []
for year in range(start_year, end_year + 1):
    avg_ben_all_cds_list.append(ColumnDataSource(avg_ben_all[year]))

In [17]:
aba_plots = []
yr = 2014
for item in avg_ben_all_cds_list:
    title = 'Average Benefit by AGI Percentile for {}'.format(yr)
    aba_plots.append(avg_ben(title, item))
    yr += 1

In [18]:
show(column(aba_plots))

# Average Benefits for Participants

In [19]:
# create list participation rate CDS's
avg_ben_part_cds_list = []
for year in range(start_year, end_year + 1):
    avg_ben_part_cds_list.append(ColumnDataSource(avg_ben_participants[year]))

In [20]:
abp_plots = []
yr = 2014
for item in avg_ben_part_cds_list:
    title = 'Average Benefit for Participants by AGI Percentile for {}'.format(yr)
    f = avg_ben(title, item)
    f.legend.orientation = 'horizontal'
    f.legend.location = 'top_center'
    f.width = 800
    f.y_range = Range1d(0, 45000)
    abp_plots.append(f)
    yr += 1

In [21]:
show(column(abp_plots))

# Sample Reforms

In [22]:
# baseline calculator
pol = Policy()
rec = Records.cps_constructor()
calc = Calculator(policy=pol, records=rec)
calc.advance_to_year(2018)
calc.calc_all()

You loaded data for 2014.
Tax-Calculator startup automatically extrapolated your data to 2014.


## Repeal SSI, SNAP, Other Benefits

In [23]:
# reform calculator
pol2 = Policy()
reform = {
    2018: {'_BEN_Benefit_switch': [[False, False, True, True, True, True, False]]}
}
pol2.implement_reform(reform)
rec2 = Records.cps_constructor()
calc2 = Calculator(policy=pol2, records=rec2)
calc2.advance_to_year(2018)
calc2.calc_all()

You loaded data for 2014.
Tax-Calculator startup automatically extrapolated your data to 2014.


In [24]:
baseline_benefits = (calc.records.ben_total * calc.records.s006).sum()
reform_benefits = (calc2.records.ben_total * calc2.records.s006).sum()
change = reform_benefits - baseline_benefits
print ('Baseline Benefits: {:18.2f}'.format(baseline_benefits))
print ('Reform Benefits: {:20.2f}'.format(reform_benefits))
print ('-' * 37)
print ('Change in Benefits: {:17.2f}'.format(change))

Baseline Benefits:   2163269869190.04
Reform Benefits:     2020391727641.31
-------------------------------------
Change in Benefits:  -142878141548.73


## Apply Welfare Multiples

In [25]:
# reform calculator
pol3 = Policy()
reform2 = {
    2018: {
        '_BEN_Vet_mult': [0.95],
        '_BEN_Medicaid_mult': [0.30],
        '_BEN_Medicare_mult': [0.75]
    }
}
pol3.implement_reform(reform2)
rec3 = Records.cps_constructor()
calc3 = Calculator(policy=pol3, records=rec3)
calc3.advance_to_year(2018)
calc3.calc_all()

You loaded data for 2014.
Tax-Calculator startup automatically extrapolated your data to 2014.


In [26]:
total_benefits = (calc3.records.ben_total * calc3.records.s006).sum()
mult_benefits = (calc3.records.ben_welfare * calc3.records.s006).sum()
change = mult_benefits - total_benefits
print ('Total Benefits: {:26.2f}'.format(total_benefits))
print ('Benefits After Multiples: {:.2f}'.format(mult_benefits))
print ('-' * 46)
print ('Difference: {:30.2f}'.format(change))

Total Benefits:           2163269869190.04
Benefits After Multiples: 1737091861659.44
----------------------------------------------
Difference:               -426178007530.60
